*note sur ce qu'il faut qu'Aurore fasse *:
- Trouver des meilleurs titre de partie
- Faire un algo RSA qui marche bien
- Faire le schéma recap en anglais
- parler de la taille des clés : "Les nombres pris ici sont volontairement petits pour un souci de compré- hension et lisibilité mais, dans la réalité, les clés utilisées et recommandées pour une sécurité optimale sont de 2048 bits, voire 4096 bits pour un usage sensible."
- improve my English level lol
- dormir zzzzzz

# RSA encryption

**RSA encryption** is an asymitric cryptosystem created by three searchers of the MIT, Ronald **R**ivest, Adi **S**hamir and Leonard **A**dleman in 1977. They were looking for a way to proove that public-key cryptography was flawed. They finally found a system which is still used nowadays for confidential communications and safe authentification online.   

## I - Principle

### How does it works

The cryptosystem is based on arithmetical propreties such as Fermat's Little Theorem, Gauss theorem or modulus computations, and an efficient combinaisons of algorithms including the Euclid's Algorithm and the Fast Exponentiation algorithm. It also uses the many propreties of prime numbers, showing that these numbers are always a powerful tool to use in cryptographie due to the many problems about them that are unsolve (Goldbach conjecture for instance).

By beeing a public-key cryptosystem, the RSA encryption uses three keys : 
- __two public keys__ to encrypt called $n$ and $e$ ($n$ is also use to decrypt) known by absolutely everybody
- __one private key__ to decrypt, $d$ wich have to be secret. 

To encrypt and decrypt, the function used is the modular exponentiation.


### Exemple of use : Alice and Bob (and Eve)

Now let's tell the story of Alice and Bob who want to talk discreetly to each other online. They know that, Eve, a spy, are listening to them. They have to cypher their messages before sending them. That's why by using RSA encryption Alice who wants to receive Bob's secret message created the required keys and shares the public keys $(n,e)$ to Bob (and indirectly Eve since she's spying) but keeps secret the private key $d$. Then, Bob encrypts his message $M$ with $n$ and $e$ : $$M^{e}\equiv C\mod{n}$$

And once Alice receives $C$ the encrypted message $M$, she does the reverse computation with her private key: $$C^{d}\equiv M\mod{n}$$

As Eve doesn't know the secret key to make the same computation as Alice, she can't read the message.

In [6]:
## mettre le schéma ici 

### How to compute the keys ?

In order to create the required key to cipher a message, we fisrt need to *generate two -large- prime numbers* we can called $p$ and $q$ which the product gives the first encryption/decryption key $n$. $$n = p \times q  \text{ ($p$ and $q$ are prime)}$$

Then, we need to compute a useful number $\phi(n)$ given by the Euler's phi function which gives the number of coprime numbers with $n$. Computing this function for p a prime number is easely since a prime number is coprime with $\forall a < p-1$. What makes that we have : $$\phi(n) = \phi(p\times q) = (p-1)(q-1)$$

We must get the second public key $e$, a chosen integer which have to be coprime with $\phi(n)$ and less than $\phi(n)$. It's in this step that the *Euclid's algorithm* is used. $$gdc(\phi(n),e) = 1$$

Finally, all that's left to do is to compute $d$, the secret and decryption key d, the inverse of $e\mod n$. Its means that $\exists (d,k) \in \mathbb{Z^2} $ such that $ed+k\phi(n) = 1$. We can draw a parallel with the Bézout's identity which also implies to use the *Extended Euclid's algorithm* to solve $$ed\equiv 1 \mod \phi(n)$$ 

We did it ! We can encrypt and decrypt quickly thanks to *Fast Exponentiation algorithm*.


## II - RSA encryption program

The best way to show what going on during the message transmission between Alice and Bob is to test by yourself with the following python program :

In [5]:
## code à bcp mieux agencer parce que c'est pas ce que je veux :
# Disclaimer : don't use for security purpose !

import binascii
import math

#algorithme d'euclide étendu pour la recherche de l'exposant secret
def euclidExtended(a,b):
    (u, v, new_u, new_v) = (1, 0, 0, 1)
    (B, A) = (b, a)

    while b != 0:
        (q, r) = divmod(a, b)  # (y // b, y % b)
        (a, b) = (b, r)
        (a, v, new_u, new_v) = (new_u, new_v, u - q * new_u, v - q * new_v)

    if B<A:
        return v%B
    else:
        return u%A

# To convert string to int 
def string_to_int(x):
    to_int =0
    for i in reversed(range(len(x))):
        to_int =int(ord(x[i]))*pow(2,(8*i))+to_int
    return(to_int)


def int_to_string(x): # pour transformer un int en string
    text=''
    tmp1=x
    while tmp1>0:
        res=tmp1%(pow(2,8))
        tmp1=(tmp1-res)//(pow(2,8))
        text=text+chr(res)
    return text


def char10_message(): #entrer le secret
    secret_message=input("Enter a maximum 10-long-secret message : ")
    while (len(secret_message)>11):
        secret_message=input("Too long ! Enter a maximum 10-long-secret message : ")
    return(secret_message)
    

#voici les éléments de la clé d'Alice
pa=2010942103422233250095259520183 #p
qa=3503815992030544427564583819137 #q
na=pa*qa  #n
phia=((pa-1)*(qa-1))//math.gcd(pa-1,qa-1)
ea=17 #exposant public
da=euclidExtended(phia,ea) #exposant privé
#voici les éléments de la clé de bob
pb=9434659759111223227678316435911 #p
qb=8842546075387759637728590482297 #q
nb=qb*pb # n
phib=((pb-1)*(qb-1))//math.gcd(pb-1,qb-1)
eb=23 # exposants public
db=euclidExtended(phib,eb) #exposant privé

## a continuer
